In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Testing files/TCGAbiolinks/meth_filtered.csv')
train.head()

,bcr_patient_barcode,DAB1,TNNT2,C14orf181,TDRD6,RRS1,PTPRF,C6orf168,DSTYK,FAM163A,...,LRRC16A,PRKRIP1,UBN2,C8orf44,WHSC1L1,KIAA1432,AUH,ZNF189,PSMD5,MAPKAP1
0,TCGA-AC-A8OS-01,0.348891,0.514490,0.046889,0.892902,0.248609,0.389591,0.321860,0.020322,0.375141,...,0.067385,0.045655,0.038745,0.057957,0.099932,0.086901,0.046532,0.057403,0.153060,0.054532
1,TCGA-AO-A12C-01,0.088570,0.555454,0.036399,0.988181,0.176808,0.250474,0.627696,0.015763,0.553984,...,0.064177,0.028168,0.050224,0.056624,0.135825,0.136274,0.051866,0.059675,0.176739,0.043663
2,TCGA-AR-A1AO-01,0.144264,0.254331,0.051838,0.974553,0.287067,0.595715,0.353657,0.017468,0.125798,...,0.060163,0.043110,0.049949,0.047550,0.127418,0.132284,0.047841,0.058746,0.294427,0.029098
3,TCGA-BH-A1EN-01,0.717604,0.509734,0.096644,0.987554,0.139243,0.285793,0.142139,0.014955,0.479332,...,0.064779,0.129740,0.043399,0.060101,0.032513,0.089825,0.050371,0.121529,0.135495,0.049102
4,TCGA-D8-A27W-01,0.177626,0.456745,0.064679,0.773658,0.117997,0.411418,0.334776,0.016078,0.132333,...,0.092684,0.073635,0.058966,0.027224,0.225899,0.093944,0.047509,0.127037,0.097649,0.035916


In [4]:
target=train['bcr_patient_barcode']

In [5]:
train.drop(['bcr_patient_barcode'], axis=1, inplace=True)
train.head()

,DAB1,TNNT2,C14orf181,TDRD6,RRS1,PTPRF,C6orf168,DSTYK,FAM163A,ROD1,...,LRRC16A,PRKRIP1,UBN2,C8orf44,WHSC1L1,KIAA1432,AUH,ZNF189,PSMD5,MAPKAP1
0,0.348891,0.514490,0.046889,0.892902,0.248609,0.389591,0.321860,0.020322,0.375141,0.643675,...,0.067385,0.045655,0.038745,0.057957,0.099932,0.086901,0.046532,0.057403,0.153060,0.054532
1,0.088570,0.555454,0.036399,0.988181,0.176808,0.250474,0.627696,0.015763,0.553984,0.098546,...,0.064177,0.028168,0.050224,0.056624,0.135825,0.136274,0.051866,0.059675,0.176739,0.043663
2,0.144264,0.254331,0.051838,0.974553,0.287067,0.595715,0.353657,0.017468,0.125798,0.458625,...,0.060163,0.043110,0.049949,0.047550,0.127418,0.132284,0.047841,0.058746,0.294427,0.029098
3,0.717604,0.509734,0.096644,0.987554,0.139243,0.285793,0.142139,0.014955,0.479332,0.751447,...,0.064779,0.129740,0.043399,0.060101,0.032513,0.089825,0.050371,0.121529,0.135495,0.049102
4,0.177626,0.456745,0.064679,0.773658,0.117997,0.411418,0.334776,0.016078,0.132333,0.317759,...,0.092684,0.073635,0.058966,0.027224,0.225899,0.093944,0.047509,0.127037,0.097649,0.035916


In [6]:
train = train.apply(lambda x : x.fillna(x.mean()), axis = 0)

In [7]:
train_scaled = minmax_scale(train, axis = 0)

In [8]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [9]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

# this is the size of our encoded representations
encoding_dim = 16

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

(14, 16474)
(2, 16474)
Epoch 1/50
2/2 - 2s - loss: 0.6932 - val_loss: 0.6931 - 2s/epoch - 977ms/step
Epoch 2/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 105ms/epoch - 52ms/step
Epoch 3/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 93ms/epoch - 46ms/step
Epoch 4/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 128ms/epoch - 64ms/step
Epoch 5/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 84ms/epoch - 42ms/step
Epoch 6/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 99ms/epoch - 50ms/step
Epoch 7/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 137ms/epoch - 69ms/step
Epoch 8/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 95ms/epoch - 47ms/step
Epoch 9/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 119ms/epoch - 60ms/step
Epoch 10/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 120ms/epoch - 60ms/step
Epoch 11/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 89ms/epoch - 44ms/step
Epoch 12/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6931 - 117ms/epoch - 58ms/step
Epoch 13/50
2/2 - 0s 

In [10]:
encoded_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
0,0.242960,0.000000,0.430528,0.508233,0.042642,0.0,0.000000,0.000000,0.000000,0.019251,0.317400,0.0,0.000000,0.419141,0.048590,0.000000
1,0.404090,0.000000,0.257471,0.000000,0.000000,0.0,0.064688,0.334051,0.589819,0.325472,1.185595,0.0,0.853618,0.682315,0.000000,0.266621
2,0.000000,0.000000,0.285691,0.000000,0.163292,0.0,0.000000,0.119644,0.178050,0.000000,0.125611,0.0,0.562978,0.337269,0.000000,0.000000
3,0.637903,0.208365,0.174785,0.000000,0.544560,0.0,0.014031,1.470044,0.646406,0.000000,1.159907,0.0,0.687400,0.560110,0.000000,0.424598
4,0.242354,0.000000,0.211337,0.000000,0.030281,0.0,0.145547,0.198089,0.127412,0.075726,0.621013,0.0,0.380107,0.688905,0.000000,0.160894
5,0.057289,0.078471,0.000000,0.000000,0.063270,0.0,0.226087,0.437817,0.391613,0.172250,0.770771,0.0,0.590269,0.654388,0.000000,0.154778
6,0.673805,0.170679,0.246609,0.000000,0.185924,0.0,0.108407,0.885395,0.624751,0.333829,1.110233,0.0,0.784917,0.542542,0.000000,0.263702
7,0.008161,0.000000,0.355795,0.000000,0.035378,0.0,0.183819,0.141383,0.317324,0.091054,0.509558,0.0,0.611306,0.627551,0.000000,0.000000
8,0.285355,0.000000,0.181587,0.000000,0.170201,0.0,0.037967,0.631973,0.552510,0.221105,0.969538,0.0,0.765655,0.427981,0.000000,0.347370
9,0.109706,0.000000,0.000000,0.000000,0.631096,0.0,0.447584,0.904266,0.538791,0.273912,1.028416,0.0,0.591538,0.430617,0.000000,0.227502


In [11]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16474)]           0         
                                                                 
 dense (Dense)               (None, 64)                1054400   
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 32)                544       
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dense_5 (Dense)             (None, 16474)             107081

In [12]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean()>0.8])
encoded_train.shape

(16, 13)

In [13]:
encoded_train

,feature_0,feature_1,feature_2,feature_4,feature_6,feature_7,feature_8,feature_9,feature_10,feature_12,feature_13,feature_14,feature_15
0,0.242960,0.000000,0.430528,0.042642,0.000000,0.000000,0.000000,0.019251,0.317400,0.000000,0.419141,0.048590,0.000000
1,0.404090,0.000000,0.257471,0.000000,0.064688,0.334051,0.589819,0.325472,1.185595,0.853618,0.682315,0.000000,0.266621
2,0.000000,0.000000,0.285691,0.163292,0.000000,0.119644,0.178050,0.000000,0.125611,0.562978,0.337269,0.000000,0.000000
3,0.637903,0.208365,0.174785,0.544560,0.014031,1.470044,0.646406,0.000000,1.159907,0.687400,0.560110,0.000000,0.424598
4,0.242354,0.000000,0.211337,0.030281,0.145547,0.198089,0.127412,0.075726,0.621013,0.380107,0.688905,0.000000,0.160894
5,0.057289,0.078471,0.000000,0.063270,0.226087,0.437817,0.391613,0.172250,0.770771,0.590269,0.654388,0.000000,0.154778
6,0.673805,0.170679,0.246609,0.185924,0.108407,0.885395,0.624751,0.333829,1.110233,0.784917,0.542542,0.000000,0.263702
7,0.008161,0.000000,0.355795,0.035378,0.183819,0.141383,0.317324,0.091054,0.509558,0.611306,0.627551,0.000000,0.000000
8,0.285355,0.000000,0.181587,0.170201,0.037967,0.631973,0.552510,0.221105,0.969538,0.765655,0.427981,0.000000,0.347370
9,0.109706,0.000000,0.000000,0.631096,0.447584,0.904266,0.538791,0.273912,1.028416,0.591538,0.430617,0.000000,0.227502


In [15]:
#encoded_train.to_excel('/content/drive/MyDrive/Testing files/TCGAbiolinks/dae_meth.xlsx', index=False)